# Group members

 - Flavia Morena Valentina Pacheco Jaime
 - Jozek Andrzej Hajduk Sánchez
 - Jorge Andres Rangel Zapata

# Connection with POSTGRESQL

In [ ]:
conn = psycopg2.connect( host = 'localhost', port = 5432, database = 'Adventure', user = 'jorgedevops' )

# Getting the data

### Table 1 Employees per area

In [ ]:
cur = conn.cursor()
table1 = pd.read_sql_query( 'SELECT st.name , count(pp.businessentityid) FROM sales.salesterritory st, sales.salesorderheader so, person.person pp WHERE pp.businessentityid = so.customerid AND st.territoryid  = so.territoryid group by (st.name)', conn )
cur.close()

_Data types table 1_

In [ ]:
table1['name']= table1['name'].astype(str)
table1.dtypes

In [ ]:
table1

### Table 2 Units per subcategory

In [ ]:
cur = conn.cursor()
table2 = pd.read_sql_query( 'SELECT  pp.name, count(ps.productsubcategoryid) FROM production.product pp, production.productsubcategory ps group by (pp.name)', conn )
cur.close()

In [ ]:
table2

### Table 3 Units per product

In [ ]:
cur = conn.cursor()
table3 = pd.read_sql_query( 'SELECT PP.NAME, PI.QUANTITY FROM PRODUCTION.PRODUCT PP JOIN PRODUCTION.PRODUCTINVENTORY PI ON PP.PRODUCTID = PI.PRODUCTID;' ,conn)
cur.close()

In [ ]:
table3

### Table 4 Employees per department

In [ ]:
cur = conn.cursor()
table4 = pd.read_sql_query( 'SELECT D.NAME, COUNT(ED.businessentityid) FROM HUMANRESOURCES.DEPARTMENT D JOIN HUMANRESOURCES.employeedepartmenthistory ED ON D.DEPARTMENTID = ED.businessentityid GROUP BY D.NAME;' ,conn)
cur.close()

In [ ]:
table4

# Connection with Google Cloud

## Credentials

In [ ]:
credentials = service_account.Credentials.from_service_account_file( 
    '/home/jorge-arch/Descargas/sixth-foundry-314518-7cb450de31bf.json', 
    scopes = [ 'https://www.googleapis.com/auth/cloud-platform' ]
)

In [ ]:
client = bigquery.Client( credentials = credentials, project = credentials.project_id )

### Table 1

In [ ]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'count', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [ ]:
job = client.load_table_from_dataframe(
    table1, 'Workshop.empleados_area', job_config = job_config
)
job.result()

### Table 2

In [ ]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'count', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [ ]:
job = client.load_table_from_dataframe(
    table2, 'Workshop.producto_por_categoria', job_config = job_config
)
job.result()

### Table 3

In [ ]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'quantity', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [ ]:
job = client.load_table_from_dataframe(
    table3, 'Workshop.productos_bogodega', job_config = job_config
)
job.result()

### Table 4

In [ ]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'count', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

job = client.load_table_from_dataframe(
    table4, 'Workshop.empleados_departamento', job_config = job_config
)
job.result()